# Imports

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalSumPool
from spektral.utils import label_to_one_hot

# Parameters

In [2]:
learning_rate = 1e-3
epochs = 10
batch_size = 32

# Load Data + play around with data

In [4]:
A, X, E, y = qm9.load_data(return_type = 'numpy', 
                          nf_keys = 'atomic_num',
                          ef_keys = 'type', 
                          self_loops = True,
                          amount = 1000)

44859392/44852087 [==============================] - 1898s 42us/step
Loading QM9 dataset.
Reading SDF


In [5]:
A #Adjacency matrix

array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 1., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 0.

In [10]:
A.shape

(1000, 8, 8)

In [6]:
X

array([[[6.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[7.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[8.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[6.],
        [7.],
        [6.],
        ...,
        [6.],
        [6.],
        [0.]],

       [[6.],
        [7.],
        [6.],
        ...,
        [6.],
        [8.],
        [0.]],

       [[6.],
        [7.],
        [6.],
        ...,
        [6.],
        [6.],
        [0.]]])

In [8]:
X.shape

(1000, 8, 1)

In [11]:
E.shape

(1000, 8, 8, 1)

In [12]:
E

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [13]:
E[0]

array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]])

In [14]:
E[999]

array([[[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.]],

       [[0.],
        [1.],
        [0.],
        [2.],
        [0.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [2.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [2.],
        [0.],
        [0.]],

       [[0.],
        [1.],
        [0.],
        [0.],
        [2.],
        [0.],
        [1.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]])

In [15]:
y

,mol_id,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,gdb_996,3.96414,3.232380,1.813300,1.7250,63.78,-0.1817,0.0545,0.2363,691.1415,0.126890,-304.727460,-304.720443,-304.719499,-304.758236,25.772,-1458.138898,-1467.067096,-1475.365275,-1359.461225
996,gdb_997,4.06611,3.264670,1.831920,1.1867,59.45,-0.1787,0.0612,0.2400,669.7830,0.114073,-324.606050,-324.599064,-324.598120,-324.637077,25.178,-1394.223969,-1402.283067,-1409.988250,-1302.441365
997,gdb_998,3.87635,3.329070,1.832230,3.7673,62.88,-0.2105,0.0365,0.2471,691.9459,0.126291,-304.744503,-304.737403,-304.736458,-304.776340,24.819,-1468.833534,-1477.709649,-1486.007200,-1370.821648
998,gdb_999,4.05002,3.446850,1.884260,2.4520,55.05,-0.2013,0.0473,0.2485,643.5085,0.103382,-340.675814,-340.669226,-340.668282,-340.707078,23.333,-1289.397336,-1296.817002,-1303.929189,-1204.140825


In [16]:
y = y[['cv']].values

In [17]:
y

array([[ 6.469],
       [ 6.316],
       [ 6.002],
       [ 8.574],
       [ 6.278],
       [ 6.413],
       [10.098],
       [ 8.751],
       [12.482],
       [10.287],
       [11.219],
       [10.89 ],
       [14.84 ],
       [13.546],
       [12.934],
       [11.041],
       [ 9.176],
       [16.893],
       [16.561],
       [15.292],
       [20.273],
       [19.052],
       [15.312],
       [12.93 ],
       [10.398],
       [13.049],
       [11.329],
       [12.147],
       [17.447],
       [17.13 ],
       [14.988],
       [14.488],
       [15.855],
       [13.845],
       [15.954],
       [15.058],
       [13.885],
       [14.78 ],
       [19.668],
       [18.431],
       [17.888],
       [16.837],
       [16.49 ],
       [14.764],
       [15.298],
       [15.679],
       [14.696],
       [12.915],
       [21.616],
       [14.821],
       [13.371],
       [13.358],
       [12.04 ],
       [26.084],
       [25.128],
       [18.723],
       [16.963],
       [17.465],
       [18.527

# Preprocessing

In [18]:
X_uniq = np.unique(X)

In [19]:
X_uniq.shape

(5,)

In [20]:
X.shape

(1000, 8, 1)

In [23]:
X_uniq  = X_uniq[X_uniq!=0] #List of all atoms in the matrix

In [25]:
E_uniq = np.unique(E)

In [26]:
E_uniq = E_uniq[E_uniq!=0]

In [27]:
E_uniq # Single, double and triple bonds?

array([1., 2., 3.])

In [28]:
X = label_to_one_hot(X, X_uniq)

In [30]:
X.shape

(1000, 8, 4)

In [31]:
X

array([[[1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [1., 0., 0., 0.],
        [1., 0.

In [33]:
E = label_to_one_hot(E, E_uniq)

In [34]:
E.shape

(1000, 8, 8, 3)

## Comments


Here I notice the first difference between this data and the one I have. Their edges and nodes are one hot feature vectors which makes sense as they come from different classes. 
Mine however are just floats. 

Orrr does it make sense to convert to one-hot into the 16 classes that are allowed for node and edge weights?
That will make my feature vectors large though. 

In that case how will my Generator work? How will it know to generate a set of nodes with one hot, and a set of edges with one hot?

Will think about this. 

# Misc (Parameters and train, test split)

In [35]:
N = X.shape[-2]
F = X[0].shape[-1]
S = E[0].shape[-1]
n_out = y.shape[-1]

In [36]:
print("%d %d %d %d" % (N, F, S, n_out))

8 4 3 1


In [38]:
A_train, A_test, X_train, X_test, E_train, E_test, y_train, y_test \
= train_test_split(A, X, E, y, test_size = 0.1, random_state = 42)

In [40]:
A_train.shape

(900, 8, 8)

# Building Model

In [41]:
X_in = Input(shape = (N, F))
A_in = Input(shape = (N, N))
E_in = Input(shape = (N, N, S))

In [42]:
X_1 = EdgeConditionedConv(32, activation='relu')([X_in, A_in, E_in])
X_2 = EdgeConditionedConv(32, activation='relu')([X_1, A_in, E_in])
X_3 = GlobalSumPool()(X_2)
output = Dense(n_out)(X_3)

In [44]:
model = Model(inputs = [X_in, A_in, E_in], outputs = output, name = "qm9_network")
optimizer = Adam(lr = learning_rate)
model.compile(optimizer=optimizer, loss = 'mse')
model.summary()

Model: "qm9_network"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8, 4)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8, 8)]       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 8, 8, 3)]    0                                            
__________________________________________________________________________________________________
edge_conditioned_conv (EdgeCond (None, 8, 32)        672         input_1[0][0]                    
                                                                 input_2[0][0]          

# Fit Model

In [47]:
model.fit([X_train, A_train, E_train], y_train, batch_size = batch_size, epochs = epochs, verbose = 2)

Epoch 1/10
29/29 - 0s - loss: 12.5929
Epoch 2/10
29/29 - 0s - loss: 11.8740
Epoch 3/10
29/29 - 0s - loss: 10.8781
Epoch 4/10
29/29 - 1s - loss: 10.3585
Epoch 5/10
29/29 - 0s - loss: 9.5261
Epoch 6/10
29/29 - 0s - loss: 9.0224
Epoch 7/10
29/29 - 0s - loss: 8.1925
Epoch 8/10
29/29 - 0s - loss: 7.5779
Epoch 9/10
29/29 - 0s - loss: 7.0301
Epoch 10/10
29/29 - 0s - loss: 6.5840


# Evaluate Model


In [48]:
print("Testing Model")
model_loss = model.evaluate([X_test, A_test, E_test], y_test, batch_size = batch_size)
print('Done. Test loss: {}'.format(model_loss))

Testing Model
4/4 [==============================] - 0s 10ms/step - loss: 7.3376
Done. Test loss: 7.33755350112915
